In [12]:
!set CMAKE_ARGS=-DLLAMA_CUBLAS=on
!set FORCE_CMAKE=1
!pip install llama-cpp-python -qq

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
!pip install -q langchain
!pip install -q unstructured
!pip install -q sentence_transformers
!pip install -q chromadb
!pip install langchain_community -qq

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [14]:
import pandas as pd
from torch import cuda
from langchain.text_splitter import  RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredExcelLoader


from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

from langchain.embeddings import HuggingFaceEmbeddings

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
print(device)

cuda:0


In [15]:
import re
import glob

books_path = "/kaggle/input/malawi-data/MW_TGBookletsExcel/MWTGBookletsExcel/*.xlsx"
booklets = glob.glob(books_path)

def remove_unwanted(text):
    pattern = re.compile(r'\\ufdd0|[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U000024C2-\U0001F251]+', flags=re.UNICODE)
    clean_text = pattern.sub('', text)
    
    return clean_text


In [16]:
for booklet in booklets:
    print(booklet)
    df= pd.read_excel(booklet, names=['row', 'text'])
    df['text'] = df['text'].apply(lambda x: remove_unwanted(str(x)))
    df.to_excel(booklet.split("/")[-1], index=False)

/kaggle/input/malawi-data/MW_TGBookletsExcel/MWTGBookletsExcel/TG Booklet 2 Sections 1,2,3_final_04112021.xlsx
/kaggle/input/malawi-data/MW_TGBookletsExcel/MWTGBookletsExcel/TG Booklet 1 Introduction Module Booklet 1TG_final_04112021.xlsx
/kaggle/input/malawi-data/MW_TGBookletsExcel/MWTGBookletsExcel/TG Booklet 3 Section 4,5,6 ,7_final_04112021.xlsx
/kaggle/input/malawi-data/MW_TGBookletsExcel/MWTGBookletsExcel/TG Booklet 5 Section 10_final_04112021.xlsx
/kaggle/input/malawi-data/MW_TGBookletsExcel/MWTGBookletsExcel/TG Booklet 6_Section 11_final_04112021.xlsx
/kaggle/input/malawi-data/MW_TGBookletsExcel/MWTGBookletsExcel/TG Booklet 4 Sections 8, 9_final_04112021.xlsx


In [17]:
cleaned_booklets = glob.glob("/kaggle/working/*.xlsx")
loaders = [UnstructuredExcelLoader(f"{booklet}", engine="") for booklet in cleaned_booklets]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [18]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                     chunk_overlap=20)
chunked_documents = text_splitter.split_documents(docs)


In [19]:
!huggingface-cli download TheBloke/Llama-2-7B-Chat-GGUF llama-2-7b-chat.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf to /root/.cache/huggingface/hub/tmpw5eu5y8h
llama-2-7b-chat.Q4_K_M.gguf: 100%|██████████| 4.08G/4.08G [00:17<00:00, 234MB/s]
./llama-2-7b-chat.Q4_K_M.gguf


In [50]:
from langchain.llms import LlamaCpp
lcpp_llm = LlamaCpp(
    model_path="/kaggle/working/llama-2-7b-chat.Q4_K_M.gguf",
    n_threads=4, # CPU cores
    n_ctx=2048,
    verbose=True,
    n_gpu_layers=-1,
    main_gpu = 1
    )


/opt/conda/lib/python3.10/site-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! main_gpu is not default parameter.
                main_gpu was transferred to model_kwargs.
                Please confirm that main_gpu is what you intended.
  warnings.warn(
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /kaggle/working/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              =

In [26]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": device}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
vectordb = Chroma.from_documents(documents=chunked_documents, embedding=embeddings, persist_directory="chroma_db")

In [51]:
retriever = vectordb.as_retriever(k=5)

qa = RetrievalQA.from_chain_type(
    llm=lcpp_llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True,
)


In [52]:
import time
def test_rag(qa, query):
    #print(f"Query: {query}\n")
    time_1 = time.time()
    result = qa.run(query)
    time_2 = time.time()
    time_taken = round(time_2-time_1, 3)

    return result,time_taken

In [53]:
test = pd.read_csv("/kaggle/input/malawi-data/Test.csv")

In [ ]:
%%time
from tqdm import tqdm

results = []
sources = []
i=0
for question in tqdm(test["Question Text"]):

    print(question)
    result,_ = test_rag(qa, question)
    docs = vectordb.similarity_search(result) # can try question instead of answers
    source = docs[0].metadata['source'].split("/")[-1]

    results.append(result)
    sources.append(source)


  0%|          | 0/499 [00:00<?, ?it/s]

What is the definition of "unusual event"


> Entering new RetrievalQA chain...
